# Demos

This is the notebook containing the demos for Feature Store, Model Monitor, and Clarify. Testing for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables that are used throughout the demos. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [ ]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()

## Feature Store
---

Feature store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this demo we'll use the boston housing dataset, which you can learn more about here: https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [ ]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=0.1, seed=1234)

# Manually add headers
train_headers = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]
test_headers = ["MEDV"]

In [ ]:
import pandas as pd
import time
import uuid


boston_train = pd.DataFrame(x_train, columns=train_headers)

Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [ ]:
boston_train["EventTime"] = time.time()
boston_train["id"] = range(len(boston_train))

# Create feature group
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name="boston-features", sagemaker_session=session
)

# Load Feature definitions
feature_group.load_feature_definitions(data_frame=boston_train)

The feature group is not created until we call the `create` method, let's do that now:

In [ ]:
feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name='id',
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

For applications, we can create a lightweight client to retrieve data with low latency:

In [ ]:
runtime = session.boto_session.client(
  'sagemaker-featurestore-runtime',
  region_name=region
)

data = runtime.get_record(
    FeatureGroupName="boston-features",
    RecordIdentifierValueAsString="0"
)

If we try to get records before we ingest any data, the response comes back empty:

In [ ]:
data

In [ ]:
feature_group.ingest(data_frame=boston_train, max_workers=3, wait=True)

In [ ]:
data = runtime.get_record(
    FeatureGroupName="boston-features",
    RecordIdentifierValueAsString="0"
)
data

## Model Monitor

In this demo we create a monitoring schedule for a deployed model. We'll begin by reloading our data from the previous demo.

In [ ]:
from tensorflow.keras.datasets import boston_housing
import pandas as pd

(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=0.1, seed=1234)
headers = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]


train = pd.DataFrame(x_train, columns=headers)
train["MEDV"] = y_train

# Target variable must come first per https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html
train.set_index(train.pop('MEDV'), inplace=True)
train.reset_index(inplace=True)
train

In [ ]:
test =  pd.DataFrame(x_test)
test["MEDV"] = y_test

# Target variable must come first per https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html
test.set_index(test.pop('MEDV'), inplace=True)
test.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data, then train a model:

In [ ]:
train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_output_location = f"s3://{bucket}/models/boston_model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

model.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)


model.fit({'train': s3_input_train, 'validation': s3_input_validation})

Now that the training job has finished, we can configure a deployment for data capture, then deploy:

In [ ]:
capture_uri = f's3://{bucket}/data-capture'

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=capture_uri
)

In [ ]:
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)

Here we can provide some sample code to test the deployed model:

In [ ]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
inputs = test.copy()
inputs = inputs.drop(columns=inputs.columns[0])

x_pred = xgb_predictor.predict(inputs.sample(5).values).decode('utf-8')

In [ ]:
s3_input_train

We define the Model Monitor and suggest a baseline:

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

In [ ]:
my_monitor.suggest_baseline(
    baseline_dataset=f's3://{bucket}/data/train.csv',
    dataset_format=DatasetFormat.csv(header=False),
)

Lastly, the Model Monitor must be scheduled, or it won't actually run regular processing jobs on the captured data:

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator

my_monitor.create_monitoring_schedule(
    monitor_schedule_name='my-monitoring-schedule',
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

## Clarify

This Clarify demo builds on the previous demo: we follow the same pattern of define-configure-schedule for our Monitor. Clarify, however, needs more config. We define `SHAPConfig`, `ModelConfig`, `ExplainabilityAnalysisConfig`, and pass them all to the scheduling method.

In [ ]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)


shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[train.mean().astype(int).to_list()[1:]],
    num_samples=int(x_train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)


model_config = sagemaker.clarify.ModelConfig(
    model_name="xgboost-2021-08-25-15-19-33-499",
    instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="text/csv",
    accept_type="text/csv",
)

analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
        explainability_config=shap_config,
        model_config=model_config,
        headers=train.columns.to_list()[1:],
    )

explainability_uri = f"s3://{bucket}/model_explainability"
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

In [ ]:
model_explainability_monitor.delete_monitoring_schedule()

In [ ]:
my_monitor.delete_monitoring_schedule()